# Make a COPY before you run the script. The data generated by python would be copied in to this excell file.

In [1]:
import pandas as pd
from excel_manager import ExcelManager
from run_data_frame_calculations import RunDataFrameCalculationsForOneDataFrame
from nice_functions import NiceExcelFunction


In [2]:
# give the path of the Excel you want to run
path_input_excel = r"C:\Users\robbe\Downloads\Gas_production_KRA_BEP_R.Grobben_final_01_run_File.xlsx"

# make instance of Excel manger
# This class is build to make it easier to do operations with the Excell
manager = ExcelManager(path_input_excel)

# load the workbook in
manager.load_workbook()

### You can see the sheet names

In [3]:
# check the sheet names
sheet_names = manager.get_sheet_names()
print(sheet_names)

['Notes', 'GT1.1', 'GT1.2', 'GT2.1', 'GT2.2', 'GT3.1', 'GT3.2', 'GT4.1', 'GT4.2', 'GT5.1', 'GT5.2', 'GT6.1', 'GT6.2', 'GT7.1', 'GT7.2', 'GT8.1', 'GT8.2']


#### input names of table if needed

In [4]:
basic_column_names = ['Sample ID', 'Parallel', 'Date', 'Time', 'P atm [hPa]', 'P sample before gc [hPa]', 'P sample after gc [hPa]', 'CH4 [%]', 'CO2 [%]', 'O2 [%]', 'N2 [%]', 'Flush (1=yes; 0=no)', 'Comments', 'GC method', 'Weight [g]']



##### you can check if the sheets are in the instance of the excell manager class.

In [5]:
# check if you can see the data_frame. With manger.load_sheet_table you can see the full sheet as a panda data frame when start_row=0.
data_frame_full_sheet = manager.load_sheet_table(sheet_name="GT1.1", start_row=0)

display(data_frame_full_sheet)
print(f' As you can see now the full sheet is shown')

,None,constant,Rgas,8314.5,Lpa/Kmol,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
0,None,constant,expTemp,293.15,K,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,volume_headspace,0.961,l,None,None,None,None,None,...,None,None,DM ?,None,None,None,None,None,None,None
2,None,constant,MM_C,12,g/mol,None,?,Molar Mass C,None,None,...,None,None,Dry Mass,None,None,None,None,None,None,None
3,None,None,water_volume,0.098,l,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,None,None,dry_mass_sample,153.607785,g,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,None,constant,henryeff_20,0.00523,None,None,for which liquid?,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,Sample ID,Parallel,Date,Time,P atm,P sample before gc,P sample after gc,CH4,CO2,O2,...,CO2_a_aq,CO2_a_aq,CO2_dissolved between timesteps_aq,CO2_cum_aq,DIC_cum,Ctot_DM,Ratio O2/CO2,None,None,None


 As you can see now the full sheet is shown


# Validate the input data with the other Notebook

### Filling th dictionary for the constants of the samples  in the instance of hte excel manager class

In [6]:
# fil in the dict with constants for the samples
sheet_names_samples = manager.get_sheet_names()
for sheet_name in sheet_names_samples:
    # loading the panda data frame for the sample
    constants_data_frame = manager.load_constants_as_data_frame(sheet_name=sheet_name, start_row=1, end_row=7, start_col=3)

    constants_data_class = manager.load_constants_as_data_class(sheet_name=sheet_name, start_row=1, end_row=7, start_col=3)

    # fill the dictionary
    manager.fill_dict_constants_data_frames(data_frame=constants_data_frame, sheets=[sheet_name])

    manager.fill_dict_constants_data_classes(data_class=constants_data_class, sheets=[sheet_name])

    print(f"loading {sheet_name} is done")

dict_constants = manager.get_dict_constants_data_frames()
dict_constants_data_classes = manager.get_dict_constants_data_classes()


loading Notes is done
loading GT1.1 is done
loading GT1.2 is done
loading GT2.1 is done
loading GT2.2 is done
loading GT3.1 is done
loading GT3.2 is done
loading GT4.1 is done
loading GT4.2 is done
loading GT5.1 is done
loading GT5.2 is done
loading GT6.1 is done
loading GT6.2 is done
loading GT7.1 is done
loading GT7.2 is done
loading GT8.1 is done
loading GT8.2 is done


You can see below the numbers of the constants of the sample loaded in the dictionary as data classes

In [7]:
data_class = dict_constants_data_classes["GT1.1"]
print(data_class.Rgas)
print(data_class.expTemp)
print(data_class.volume_headspace)
print(data_class.water_volume)
print(data_class.dry_mass_sample)

8314.5
293.15
0.961
0.098
153.60778531406666


#### Run all the calculations for each data frame (each data frame = one sample)

While using a for loop all the calculations are operated for each data frame in the dictionary. Go to the specific class and method if you want to see how te calculation is done.

The calculations are only done based on the raw-input data.

In [8]:
# Now all the data frames are read in, calculated and returned in to the excell_file
sheet_names_samples = sheet_names[1:]
for sheet_name in sheet_names_samples:
    print(f"Started with sheet: {sheet_name}")
    # loading the data frame
    column_end = NiceExcelFunction.get_column_index_from_letter(column_letter="O") #The end column of the Excel is at 0
    # The table starts at row 12.
    data_frame = manager.load_sheet_table_with_input_header(sheet_name=sheet_name, column_names=basic_column_names, start_row=12, end_column=column_end)

    # loading the data class with the constants for the sample
    constants_data_class = dict_constants_data_classes[sheet_name]

    # adding date and do the gas composition corrections.
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_data_frame_processor_calculations()
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_gas_composition_calculations(set_values_gas_composition_first_row=True,
                                                                                                    ch4=0, co2=0.03, o2=21.90, n2=78.07)

    # calculations for moles before and after sampling
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_mol_gases_before_and_after_sampling(Rgas=constants_data_class.Rgas,
                                                                                                           exp_temperature=constants_data_class.expTemp,
                                                                                                           volume_headspace=constants_data_class.volume_headspace)
    # correcting the mg_bs for the first measurement
    data_frame.loc[0, "mg_as"] = data_frame.loc[0, "mg_bs"]

    # run mol gas composition
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_mol_gas_composition_calculation()
    # run moles produced
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_moles_produced()
    # run cumulative calculations carbon in gas phase
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_cumulative_production_in_the_gas_phase(molar_mass_carbon=constants_data_class.molar_mass_carbon,
                                                                                                              dry_mass_sample=constants_data_class.dry_mass_sample)

    # run calculation carbon in the aqueous phase
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_carbon_in_aqueous_phase(water_volume_in_liters=constants_data_class.water_volume,
                                                                                               dry_mass_sample=constants_data_class.dry_mass_sample)

    # run results interpretations
    RunDataFrameCalculationsForOneDataFrame(data_frame=data_frame).run_results_Interpretations()

    # print to see the process
    print(f"Finished calculations {sheet_name}")

    # overwrite the old data frame or fill with the new data frame which has calculations
    manager.fill_dict_panda_data_frames(data_frame=data_frame, sheets=[sheet_name])

    # print to see the process
    print(f"Filled dictionary for {sheet_name}")

started with sheet: GT1.1
finished calculations GT1.1
filled dictionary for GT1.1
started with sheet: GT1.2
finished calculations GT1.2
filled dictionary for GT1.2
started with sheet: GT2.1
finished calculations GT2.1
filled dictionary for GT2.1
started with sheet: GT2.2
finished calculations GT2.2
filled dictionary for GT2.2
started with sheet: GT3.1
finished calculations GT3.1
filled dictionary for GT3.1
started with sheet: GT3.2
finished calculations GT3.2
filled dictionary for GT3.2
started with sheet: GT4.1
finished calculations GT4.1
filled dictionary for GT4.1
started with sheet: GT4.2
finished calculations GT4.2
filled dictionary for GT4.2
started with sheet: GT5.1
finished calculations GT5.1
filled dictionary for GT5.1
started with sheet: GT5.2
finished calculations GT5.2
filled dictionary for GT5.2
started with sheet: GT6.1
finished calculations GT6.1
filled dictionary for GT6.1
started with sheet: GT6.2
finished calculations GT6.2
filled dictionary for GT6.2
started with she

#### You can see the mentioned calculated data frame below. Please fill the name the sample in the "sheet_tab_name" to get the table you want to see.

In [9]:
sheet_tab_name = "GT1.1"

dict_data_frames = manager.get_dict_panda_data_frames()
display(dict_data_frames[sheet_tab_name])
# Set the display options
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', None)  # Automatically adjust the column width to fit the contents

,Sample ID,Parallel,Date,Time,P atm [hPa],P sample before gc [hPa],P sample after gc [hPa],CH4 [%],CO2 [%],O2 [%],...,CO2_b_aq [mol/m3],CO2_b_aq [mol],PP CO2_a,CO2_a_aq [mol/m3],CO2_a_aq [mol],CO2_dissolved_between_time_steps_aq,CO2_produced_aq_cum,DIC_cum,Ctot_DM [mg C/gDW],Ratio O2/CO2
0,GT1,1,2023-04-17,14:42:00,1025.1,1117.44,0.00,0.00,0.030,21.900,...,0.175326,0.000017,0.000000,0.000000,0.000000,0.000017,0.000017,0.001342,0.000000,0.000000
1,GT1,1,2023-04-19,14:42:00,1023.1,960.00,958.10,0.12,1.530,2.210,...,7.643606,0.000749,1458.600000,7.628478,0.000748,0.000749,0.000766,0.059861,0.107374,6.719315
2,GT1,1,2023-04-24,10:20:00,1002.3,962.20,959.80,0.25,2.410,1.590,...,11.730204,0.001150,2237.274398,11.700945,0.001147,0.000402,0.001168,0.091263,0.166575,0.347675
3,GT1,1,2023-04-24,11:25:00,1003.3,1104.60,1085.80,0.00,0.740,19.790,...,4.388690,0.000430,824.855764,4.313996,0.000423,-0.000717,0.000452,0.035281,NaN,NaN
4,GT1,1,2023-04-26,08:52:00,1017.0,954.00,951.00,0.03,2.720,8.460,...,13.102165,0.001284,2497.316084,13.060963,0.001280,0.000861,0.001313,0.102562,0.230482,3.691717
5,GT1,1,2023-04-26,09:53:00,1017.2,1120.80,1105.60,0.00,0.630,20.510,...,3.781796,0.000371,713.290323,3.730508,0.000366,-0.000909,0.000404,0.031522,NaN,NaN
6,GT1,1,2023-04-28,09:45:00,1006.8,1029.20,1025.80,0.00,4.520,11.770,...,22.847100,0.002239,4354.038877,22.771623,0.002232,0.001873,0.002277,0.177876,0.418379,1.409119
7,GT1,1,2023-04-28,09:46:00,1006.8,1117.44,1111.45,0.00,0.538,20.313,...,3.251216,0.000319,618.315031,3.233788,0.000317,-0.001913,0.000364,0.028431,NaN,NaN
8,GT1,1,2023-05-01,13:50:00,1013.1,1042.70,1039.90,0.00,5.740,9.740,...,31.682250,0.003105,6041.524291,31.597172,0.003097,0.002788,0.003152,0.246228,0.654273,1.044409
9,GT1,1,2023-05-01,14:49:00,1013.3,1114.60,1110.60,0.00,0.780,20.500,...,4.662052,0.000457,888.206706,4.645321,0.000455,-0.002640,0.000512,0.040017,NaN,NaN


# OVERWRITING the Excel file. Have you made a copy before?

##### Uncomment those lines below to overwrite the Excel file. Please, to prevent corruption of your Excel file,  replace the "#" after you have run the cell below.

In [12]:
# for sheet_name in sheet_names_samples:
#  # replace the table (overwrite) in the Excel file.
#     data_frame = manager.get_dict_panda_data_frames()[sheet_name]
#     # the start_row is one more than in the data_frame. The data_frame starts with row 0, the Excel with 1.
#     manager.replace_table_in_specific_sheet_with_data_frame(excel_file_path=path_input_excel,
#                                                             sheet_name=sheet_name,
#                                                             start_row=13,
#                                                             data_frame=data_frame,
#                                                             header=False
#                                                             )
#     print(f"replaced table in Excel for {sheet_name}")